In [ ]:
#Recommender System - collaborative filtering  (Item Based - Recommending Items)

import numpy as np
import pandas as pd

#reading Data Set
ratings = pd.read_csv('/kaggle/input/100k-movie-rating/ratings.csv')
movies = pd.read_csv('/kaggle/input/100k-movie-rating/movies.csv')

data=pd.merge(ratings, movies, on='movieId', how='inner')

#create a rating dataset with average rating of each movie
rating= pd.DataFrame(data.groupby('title')['rating'].mean())


#Create a user-item interaction matrix by pivot table using userid as rows and titles as columns and ratings as values
matrix= data.pivot_table(index='userId', columns='title', values='rating')

#Add number of times a movie is rated in rating dataset
rating['no_of_ratings']= data.groupby('title')['rating'].count()


#get data of a desired movie from the user-item interaction matrix
pulp_fiction=matrix['Pulp Fiction (1994)']

#find similar movies by calculating the correlation with other columns in the matrix
similar=matrix.corrwith(pulp_fiction)

#create a correlation dataframe from the similar movies and drop the movies with nan
corr=pd.DataFrame(similar, columns=['correlation'])
corr.dropna(inplace=True)

#with the corr dataframe join the number of rating from rating dataset
corr=corr.join(rating['no_of_ratings'])
#filter the corr dataframe by number of ratings and sort it in descending
corr[corr['no_of_ratings']>25].sort_values(by='correlation', ascending=False).head(10)

In [ ]:
#Recommender System - collaborative filtering (User based - Finding Similar User and suggesting movies)

import numpy as np
import pandas as pd

#reading Data Set
ratings = pd.read_csv('/kaggle/input/100k-movie-rating/ratings.csv')
movies = pd.read_csv('/kaggle/input/100k-movie-rating/movies.csv')

data=pd.merge(ratings, movies, on='movieId', how='inner')

#create a rating dataset with average rating of each movie
rating= pd.DataFrame(data.groupby('title')['rating'].mean())


#Create a user-item interaction matrix by pivot table using userid as rows and titles as columns and ratings as values
matrix= data.pivot_table(index='title', columns='userId', values='rating')
m_matrix= data.pivot_table(index='userId', columns='title', values='rating')

#Add number of times a movie is rated in rating dataset
rating['no_of_ratings']= data.groupby('title')['rating'].count()


#get data of a desired movie from the user-item interaction matrix
one=matrix[3]

#find similar movies by calculating the correlation with other columns in the matrix
similar=matrix.corrwith(one)

#create a correlation dataframe from the similar movies and drop the movies with nan
corr=pd.DataFrame(similar, columns=['correlation'])
corr.dropna(inplace=True)

#filter the corr dataframe by number of ratings and sort it in descending
corr.sort_values(by='correlation', ascending=False)

similar_user_matrix=matrix[similar.index.tolist()]
similar_user_mean_rating=similar_user_matrix.mean()

highly_rated_similar_movies=similar_user_mean_rating[similar_user_mean_rating>=3.5]
highly_rated_similar_movies= matrix.loc[:, highly_rated_similar_movies.index]

similar_movies=m_matrix.corrwith(highly_rated_similar_movies.T)
similar_movies.dropna(inplace=True)


corr_movies=pd.DataFrame(similar_movies, columns=['correlation'])
corr_movies.dropna(inplace=True)

corr_movies=corr_movies.join(rating['no_of_ratings'])

recommendation=corr_movies[corr_movies['correlation']<1.0].sort_values(by='correlation',ascending=False)
print(recommendation.head(10))